In [21]:
import datetime
import json

import boto3
import requests

In [22]:
# Try different dates to see how the data changes
DATE_PARAM = "2025-11-23"

date = datetime.datetime.strptime(DATE_PARAM, "%Y-%m-%d")

# Construct the API URL
url = f"https://wikimedia.org/api/rest_v1/metrics/pageviews/top/en.wikipedia.org/all-access/{date.strftime('%Y/%m/%d')}"
print(f"Requesting REST API URL: {url}")

# Make the API request
wiki_server_response = requests.get(url, headers={"User-Agent": "curl/7.68.0"})
wiki_response_status = wiki_server_response.status_code
wiki_response_body = wiki_server_response.text

print(f"Wikipedia REST API Response body: {wiki_response_body[:500]}...")
print(f"Wikipedia REST API Response Code: {wiki_response_status}")

# Validate response
if wiki_response_status != 200:
    raise Exception(f"Received non-OK status code from Wiki Server: {wiki_response_status}")
print(f"Successfully retrieved Wikipedia data, content-length: {len(wiki_response_body)}")

Requesting REST API URL: https://wikimedia.org/api/rest_v1/metrics/pageviews/top/en.wikipedia.org/all-access/2025/11/23
Wikipedia REST API Response body: {"items":[{"project":"en.wikipedia","access":"all-access","year":"2025","month":"11","day":"23","articles":[{"article":"Main_Page","views":6168733,"rank":1},{"article":"Special:Search","views":848523,"rank":2},{"article":"Tatiana_Schlossberg","views":282949,"rank":3},{"article":"Google_Chrome","views":282854,"rank":4},{"article":"Wikipedia:Featured_pictures","views":237348,"rank":5},{"article":".xxx","views":231560,"rank":6},{"article":"Wicked:_For_Good","views":231066,"rank":7},{"article":"Caro...
Wikipedia REST API Response Code: 200
Successfully retrieved Wikipedia data, content-length: 55793


In [23]:
# Parse the API response and extract top edits
wiki_response_parsed = wiki_server_response.json()
top_views = wiki_response_parsed["items"][0]["articles"]

# Transform to JSON Lines format
current_time = datetime.datetime.now(datetime.timezone.utc)
json_lines = ""
for page in top_views[:5]:
    record = {
        "title": page["article"],
        "views": page["views"],
        "rank": page["rank"],
        "date": date.strftime("%Y-%m-%d"),
        "retrieved_at": current_time.replace(tzinfo=None).isoformat(),
    }
    json_lines += json.dumps(record) + "\n"

print(f"Transformed {len(top_views)} records to JSON Lines")
print(f"First few lines:\n{json_lines[:500]}...")

Transformed 1000 records to JSON Lines
First few lines:
{"title": "Main_Page", "views": 6168733, "rank": 1, "date": "2025-11-23", "retrieved_at": "2025-12-17T15:05:09.751836"}
{"title": "Special:Search", "views": 848523, "rank": 2, "date": "2025-11-23", "retrieved_at": "2025-12-17T15:05:09.751836"}
{"title": "Tatiana_Schlossberg", "views": 282949, "rank": 3, "date": "2025-11-23", "retrieved_at": "2025-12-17T15:05:09.751836"}
{"title": "Google_Chrome", "views": 282854, "rank": 4, "date": "2025-11-23", "retrieved_at": "2025-12-17T15:05:09.751836"}
{"ti...


In [24]:
S3_WIKI_BUCKET = "enkhsaikhan-wikidata"
s3 = boto3.client("s3")
default_region = "eu-west-1"
bucket_names = [bucket["Name"] for bucket in s3.list_buckets()["Buckets"]]

if S3_WIKI_BUCKET not in bucket_names:
    # LAB 1: Create the bucket if it doesn't exist
    bucket_configuration = {"LocationConstraint": default_region}
    response = s3.create_bucket(Bucket=S3_WIKI_BUCKET, CreateBucketConfiguration=bucket_configuration)
    print(f"Created new bucket: {S3_WIKI_BUCKET}")
else:
    print(f"Using existing bucket: {S3_WIKI_BUCKET}")

Using existing bucket: enkhsaikhan-wikidata


In [25]:
# LAB 2: Upload json_lines directly to S3
s3_key = f"raw-views/raw-views-{date.strftime('%Y-%m-%d')}.json"
s3.put_object(Bucket = S3_WIKI_BUCKET,
              Key = s3_key,
              Body = json_lines,
)


{'ResponseMetadata': {'RequestId': 'KAXJXCCSGC809DDR',
  'HostId': 'Tk3TGI1V+BW80UbVOc28H7QhpaOI2G5qmVdnv2s5h/5/77CWtvj+VMyF28I0t4mWwJLCiiENii/9eHWBpqj9S4eFh20YtMli',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'Tk3TGI1V+BW80UbVOc28H7QhpaOI2G5qmVdnv2s5h/5/77CWtvj+VMyF28I0t4mWwJLCiiENii/9eHWBpqj9S4eFh20YtMli',
   'x-amz-request-id': 'KAXJXCCSGC809DDR',
   'date': 'Wed, 17 Dec 2025 15:05:11 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"f9d539be088165e972efc375dd43b777"',
   'x-amz-checksum-crc32': '1+O+aw==',
   'x-amz-checksum-type': 'FULL_OBJECT',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"f9d539be088165e972efc375dd43b777"',
 'ChecksumCRC32': '1+O+aw==',
 'ChecksumType': 'FULL_OBJECT',
 'ServerSideEncryption': 'AES256'}